<a href="https://colab.research.google.com/github/DanielaPosadas/SQL-Mystery/blob/main/Copia_de_SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Escenario

¡Se ha cometido un crimen y los detectives necesitan tu ayuda! Te dieron el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un `murder` _(asesinato)_ que ocurrió en algún momento el 15 de Enero de 2018 y que tuvo lugar en `SQL City`. Depende de ti averiguar quién es el asesino usando solo tus habilidades en SQL y tu ingenio. Te proporcionan una conexión a la base de datos del Departamento de Policía, que tiene todas las pistas que necesitarás para atrapar al asesino.

Usa las habilidades que desarrollaste en SQL 101 y 102, junto con cualquier recurso que desees, para resolver el `SQL Murder Mystery`!"

## Conéctate a la base de datos del Departamento de Policía

Para comenzar y ejecutar tus consultas, presiona play en la celda de abajo para conectarte a la base de datos del Departamento de Policía.

Para ejecutar consultas, crea una nueva celda de `Code` y escribe `%%sql` en la parte superior. Luego puedes escribir tus consultas debajo. Ve el ejemplo a continuación:
```python
%%sql

SELECT *
FROM table

```

In [ ]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

## Descubriendo tablas en la base de datos

Comenzamos nuestra búsqueda para encontrar al asesino explorando la base de datos del Departamento de Policía. Pero aún no has visto la base de datos y no sabes cuáles son las tablas, así que, ¿cómo sabes qué buscar?

Afortunadamente, la mayoría de los sistemas de gestión de bases de datos relacionales tienen esta información almacenada en un lugar donde puedes consultarla. Muy a menudo, se utiliza un esquema especial conocido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para almacenar información sobre las tablas y columnas en tu base de datos (también conocido como metadata). La base de datos del Departamento de Policía tiene un information schema, con la vista `tables` que te muestra qué tablas están disponibles, y la vista `columns` que te muestra todas las columnas de cada tabla y sus tipos de datos.

### Listando todas las tablas en la base de datos de la Policía

#### Revisa la [descripción de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primero, veremos todas las tablas disponibles para nosotros consultando la vista `information_schema.tables`. Te daremos la primera consulta para empezar, pero de aquí en adelante tendrás que idear las consultas restantes utilizando tu conocimiento de SQL y tu ingenio.

Aquí está la consulta necesaria para mostrarte las tablas en la base de datos del Departamento de Policía. Cópiala/Pégala en la celda de abajo y ejecútala para ver las tablas disponibles para ti:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [ ]:
%%sql
SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas las tablas y sus columnas en la base de datos de la Policía

**Revisa la [descripción de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

¡Genial! Ahora que sabes qué tablas están disponibles, es hora de averiguar las columnas que tiene cada tabla junto con el tipo de datos de cada columna. Escribe una consulta que muestre el nombre de la tabla, el nombre de la columna y el tipo de datos para cada tabla en la base de datos del Departamento de Policía utilizando la vista `information_schema.columns`. Asegúrate de que la salida esté ordenada por nombre de tabla y luego por nombre de columna (ascendente):

*Pistas opcionales - ¡intenta usarlas solo si te quedas **REALMENTE** atascado!*

<details>
<summary>Pista 1</summary>
<p>Estructura tu consulta de la misma manera que lo hicimos en la consulta anterior donde miramos `information_schema.tables`</p>
</details>

<details>
<summary>Pista 2</summary>
<p>Asegúrate de revisar el enlace para la descripción de information_schema.columns. Te dirá los nombres de las columnas que debes usar para la consulta. Recuerda, queremos el nombre de la tabla, el nombre de la columna y el tipo de datos - ¡revisa la descripción para encontrar los nombres apropiados!</p>
</details>

<details>
<summary>Pista 3</summary>
<p>No olvides usar la declaración ORDER BY para ordenar los resultados de tu búsqueda. Estamos buscando ordenar ambas columnas en orden ascendente, lo cual se puede especificar usando la palabra clave ASC, sin embargo, también es el orden de clasificación predeterminado, por lo que ASC no es necesariamente requerido.</p>
</details>

In [ ]:
%%sql

UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?


### Entity Relationship Diagram (Optional)

After doing some detective work and finding the tables in the Police Department database, you discover that there is an ERD! This can be really helpful in your quest to find the murderer. If you like, challenge yourself (and your SQL skills) to continue on by just querying the `information_schema` as needed. Otherwise, you can click the dropdown below to reveal the ERD to help you see the tables and relationships in the Police Department database at a glance:

<details>
  <summary>Check the SQL Murder Mystery ERD</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

### Diagrama de Relaciones de Entidad (Opcional)

Después de hacer un poco de trabajo de detective y encontrar las tablas en la base de datos del Departamento de Policía, descubres que hay un ERD (Diagrama de Relaciones de Entidad). Esto puede ser realmente útil en tu búsqueda para encontrar al asesino. Si lo prefieres, desafíate a ti mismo (y tus habilidades en SQL) a continuar solo consultando el `information_schema` según sea necesario. De lo contrario, puedes hacer clic en el desplegable a continuación para revelar el ERD que te ayudará a ver las tablas y las relaciones en la base de datos del Departamento de Policía de un vistazo:

<details>
  <summary>Revisa el ERD del SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

In [ ]:
%%sql

___

## Resolviendo el Misterio (y verificando si resolviste el caso)

De aquí en adelante, usarás tus habilidades y las pistas en la base de datos del Departamento de Policía para encontrar al asesino. Cuando creas que has encontrado al asesino, vuelve a esta sección y usa la celda de abajo para verificar tu respuesta y ver si encontraste al asesino:

ejemplo:
```python
check_suspect("Adam Levine")
```

In [ ]:
check_suspect("Miranda Priestly")

Congrats, you found the brains behind the murder!
Everyone in SQL City hails you as the greatest SQL detective of all time.
Time to break out the champagne!


True

---

## Espacio de Trabajo

Usa las celdas de abajo para escribir tus consultas y trabajar en resolver el misterio. Cuando tengas un sospechoso, verifica tu respuesta usando la [celda arriba de esta](#solving-the-mystery-and-checking-if-you-cracked-the-case) y ejecuta su nombre a través de la función `check_suspect`. Si encuentras al asesino, la función te lo dirá.

No olvides usar `%%sql` en la parte superior de las celdas de `Code` que crees para que tus consultas SQL funcionen. ¡Buena suerte, detectives!

Para obtener un contexto completo sobre el crimen, se realizará una consulta a la base de datos del Departamento de Policía, específicamente a la tabla **crime_scene_report**. La consulta (query) incluirá todas las columnas relevantes y aplicará una cláusula **WHERE** para especificar la fecha del incidente. Además, se utilizará un operador **AND** para filtrar por la jurisdicción de la ciudad donde ocurrió el delito. Esta consulta permitirá recuperar informes detallados de la escena del crimen, facilitando el análisis de patrones criminales y la recopilación de evidencia.

In [ ]:
%%sql
SELECT *
FROM crime_scene_report
WHERE date = 20180115
AND city = 'SQL City'
AND type = 'murder'

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...,SQL City


Gracias al registro en la base de datos sobre la escena del crimen, sabíamos que teníamos dos testigos que presenciaron el asesinato. Por lo tanto, procedimos a realizar una nueva consulta a la base de datos, específicamente a la tabla interview, para intentar localizar a los posibles testigos. La consulta (query) seleccionará todas las columnas, pero se incluirá un filtro **WHERE**, seguido de un **LIKE** con la palabra clave **'murder'** aplicado a nuestra columna **transcript** para identificar entrevistas relacionadas con el homicidio.

In [ ]:
%%sql
SELECT *
FROM interview
WHERE transcript LIKE '%murder%'

,person_id,transcript
0,14849,"murder to leave it behind?’ She said the last words out loud, and the\n"
1,15121,"Queen jumped up and bawled out, “He’s murdering the time! Off with his\n"
2,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last we..."


La consulta fue exitosa, ya que aparentemente encontramos a nuestro primer testigo, quien relató que observó el asesinato. Ella identificó al sospechoso, mencionando que lo reconoció del gimnasio al que asistía, y afirmó que esto ocurrió la última vez que fue a entrenar.

Teníamos dicha declaración, sin embargo desconocíamos la identidad de la testigo, por lo que debíamos realizar una consulta a la base de datos **person** selecionando todas las columnas, aplicando un filtro **WHERE** a la columna **ID** para obtener más información sobre la testigo ocular.

In [ ]:
%%sql
SELECT *
FROM person
WHERE id = 16371

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Finalmente, nuestra primera testigo fue identificada como **Annabel Miller**. Nuestro siguiente paso era acceder a la base de datos del gimnasio para obtener pistas sobre las personas que asistieron ese día específico, dentro del mismo rango horario que Anabelle.

La consulta la realizaríamos específicamente a la base de datos del gimnasio, primero comenzando por la de miembos (**get_ git_now_member**) donde seleccionaríamos todas las columnas y filtaríamos a través de un **WHERE** en la columna **person_id** para obtener el **membership_id** de Annabel.

In [ ]:
%%sql
SELECT *
FROM get_fit_now_member
WHERE person_id = 16371

,id,person_id,name,membership_start_date,membership_status
0,90081,16371,Annabel Miller,20160208,gold


Una vez que obtuvimos el **membership_id** de Annabel Miller, accedimos a la base de datos **get_fit_now_check_in** del gimnasio para obtener los registros de entrada y salida de todos los miembros. Filtraríamos la columna membership_id utilizando una cláusula **WHERE** con el **ID** de Annabel.

In [ ]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE membership_id = '90081'

,membership_id,check_in_date,check_in_time,check_out_time
0,90081,20180109,1600,1700


Ahora teníamos claro que la última vez que la testigo asistió al gimnasio fue el 09/01/2018. Por lo tanto, la siguiente consulta se realizaría nuevamente en la misma base de datos, utilizando la cláusula **WHERE**. Sin embargo, en esta ocasión, cambiaríamos la columna membership_id por check_in_date para filtrar por la fecha en la que Annabel asistió por última vez.

In [ ]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


La base de datos arrojó varias coincidencias, ya que 10 personas habían asistido ese día. Sin embargo, nos enfocamos en nuestra testigo clave, quien era fundamental para esta investigación. Al analizar su hora de entrada y salida (check-in y check-out), descubrimos información interesante.

Nuestra testigo había ingresado al gimnasio a las 1600. Inicialmente, pensamos en descartar a todos aquellos que llegaron y se retiraron antes de esa hora. Sin embargo, surgió un detalle interesante: una de esas 10 personas llegó antes que Annabel y se retiró al mismo tiempo, por lo que decidimos no descartar completamente a esta persona. Por lo tanto, ajustamos el parámetro de la hora de llegada a las 1530.

Esta nueva información nos llevó a modificar nuestro filtro de tiempo para incluir registros desde las 1530 en adelante.

In [ ]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109
AND check_in_time BETWEEN 1530 AND 1600

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700
2,90081,20180109,1600,1700


Con dos sospechosos potenciales identificados, procedimos a revisar las declaraciones de ambos en la base de datos de entrevistas. Este paso crucial finalmente reveló la prueba contundente que necesitábamos para identificar al criminal.

In [ ]:
%%sql
SELECT *
FROM interview
WHERE person_id = 67318

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" ..."


Conocíamos el **ID** de nuestro asesino, pero aún no sabíamos su identidad. Realizamos una búsqueda en la base de datos 'person' utilizando la cláusula **WHERE** con un filtro en la columna 'id' para identificar al culpable. El nombre de nuestro asesino resultó ser **Jeremy Bowers**.

In [ ]:
%%sql
SELECT *
FROM person
WHERE id = 67318

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Ingresamos el nombre de nuestro asesino para confirmar su papel dentro del crimen.

In [ ]:
check_suspect("Jeremy Bowers")

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge,try querying the interview transcript of the murderer to find the real villain behind this crime.If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.
Use this same `check_suspect` function with your new suspect to check your answer.


True

Si bien habíamos identificado a nuestro asesino, la tarea aún no estaba completa. Según la declaración de Bowers, habíamos capturado al ejecutor del crimen, pero aún necesitábamos identificar a la mente maestra detrás de todo esto.

Sabíamos que nuestra autora intelectual era una mujer de aproximadamente 65 pulgadas de altura y con abundantes recursos económicos.

Decidimos buscar más información para obtener más pistas, ya que solo contábamos con el testimonio de una de las dos testigos y la identidad del criminal. Accediendo a otras bases de datos, encontramos una pista muy intrigante.

Dentro de la base de datos **facebook_event_check_in**, al agregar una cláusula **WHERE** en la columna **person_id**, descubrimos que Jeremy Bowers, el criminal identificado, había asistido al mismo evento que nuestra testigo principal, Annabel Miller, y un nuevo individuo que se incorporaba por primera vez a nuestra investigación.

**EVENTOS A LOS QUE ASISTIÓ JEREMY BOWERS**

In [ ]:
%%sql
SELECT *
FROM facebook_event_checkin
WHERE person_id = 67318

,person_id,event_id,event_name,date
0,67318,4719,The Funky Grooves Tour,20180115
1,67318,1143,SQL Symphony Concert,20171206


**EVENTO AL QUE ASISTIERON JEREMY BOWERS Y ANNABEL MILLER**

In [ ]:
%%sql
SELECT *
FROM facebook_event_checkin
WHERE event_name = 'The Funky Grooves Tour'

,person_id,event_id,event_name,date
0,14887,4719,The Funky Grooves Tour,20180115
1,16371,4719,The Funky Grooves Tour,20180115
2,67318,4719,The Funky Grooves Tour,20180115


Finalmente identificamos a nuestro segundo testigo ocular, **Morty Schapiro**.
Todo cobra sentido, ya que la descripción de la escena del crimen menciona la presencia de dos testigos, indicando que uno de ellos vivía en la última casa de una ubicación desconocida.
Al consultar la base de datos 'person' con una cláusula **WHERE** en la columna **ID** para obtener la identidad y dirección de nuestro testigo, confirmamos que vive en 'Northwestern Dr'. Al realizar una búsqueda adicional en la misma base de datos, esta vez en la columna **address_street_name** y aplicando **ORDER BY DESC** al **address_number**, confirmamos que efectivamente el señor Morty Schapiro reside en la última casa de esa avenida.

**IDENTIFICANDO AL SEGUNDO TESTIGO POR SU ID**

In [ ]:
%%sql
SELECT *
FROM person
WHERE id = 14887

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


**CONFIRMANDO QUE NUESTRO TESTIGO VIVE EN LA ÚLTIMA CASA DE NORTHWESTERN DR**

In [ ]:
%%sql
SELECT *
FROM person
WHERE address_street_name	= 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 5

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829


Y lo confirmamos nuevamente al ingresar el id de nuestro segundo testigo ocular a la base de datos interview para revisar su declaración de los hechos.

In [ ]:
%%sql
SELECT *
FROM interview
WHERE person_id = 14887

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership nu..."


Las piezas del rompecabezas iban encajando una por una; sin embargo, aún debíamos identificar a la segunda asesina. Resultó intrigante descubrir que Jeremy Bowers no solo había asistido a un concierto, sino a dos. Decidimos iniciar nuestra investigación por ahí, ya que la información del primer concierto había dado resultados positivos.

Realizamos una consulta a la base de datos **'facebook_event_checkin'**, aplicando una cláusula **WHERE** y filtrando la columna **'event_name'**. Además, usamos un operador **AND** para agregar un filtro en la columna **'date'**, con el objetivo de identificar a las personas que pudieron coincidir en la misma fecha y evento al que asistió nuestro asesino.

In [ ]:
%%sql
SELECT *
FROM facebook_event_checkin
WHERE event_name = 'SQL Symphony Concert'
AND date = 20171206

,person_id,event_id,event_name,date
0,69325,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171206
2,67318,1143,SQL Symphony Concert,20171206


Los resultados de nuestra investigación nos llevaron a dos sospechosos potenciales además de nuestro asesino. Sin embargo, gracias a la declaración de Jeremy Bowers, sabíamos que nuestra criminal era una mujer de aproximadamente 65 pulgadas de altura, un detalle crucial que nos permitió estrechar el cerco.

Después de consultar ambos identificadores en la base de datos 'person', nuestros corazones latían con expectativa mientras los datos revelaban el nombre de aquella mujer. Y entonces, llegamos al momento culminante de nuestra búsqueda: nuestra asesina era Miranda Priestly.


**INGRESAMOS ID E IDENFITICAMOS A UN HOMBRE -DESCARTADO-**

In [ ]:
%%sql
SELECT *
FROM person
WHERE id = 69325

,id,name,license_id,address_number,address_street_name,ssn
0,69325,Tyson Ojard,142413,3187,Fletcher Way,609595066


**INGRESAMOS ID E IDENTIFICAMOS A UNA MUJER -NUESTRA ASESINA-**

In [ ]:
%%sql
SELECT *
FROM person
WHERE id = 99716

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


**CONFIRMAMOS QUE ES ELLA CON EL DATO DE SU ESTATURA APROXIMADAMENTE 5'5 (65 PULGADAS)**

In [ ]:
%%sql
SELECT *
FROM drivers_license
WHERE id = 202298

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


**CONFIRMAMOS SUS INGRESOS ANUALES**

In [ ]:
%%sql
SELECT *
FROM income
WHERE ssn = 987756388


,ssn,annual_income
0,987756388,310000


Finalmente ingresamos su nombre en la función para confirmar definitivamente el papel de Miranda dentro del crimen.

In [ ]:
check_suspect("Miranda Priestly")

Congrats, you found the brains behind the murder!
Everyone in SQL City hails you as the greatest SQL detective of all time.
Time to break out the champagne!


True